## Prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

import statsmodels.api as sm
from scipy import stats
import statsmodels.tools as tools
from scipy.stats import bootstrap

from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 100
pd.options.display.max_columns = None

# Set a seed for NumPy
np.random.seed(0)

# Set a seed for Python's built-in random module
random.seed(0)

### Raw data

In [2]:
mesa_raw = pd.read_csv('../data_processed/MESA/Y_BaselineX_raw_full.csv')

diet_phys_map = {0:1, 1:2, 2:3}
mesa_raw['nutrition'] = mesa_raw['nutrition'].replace(diet_phys_map)
mesa_raw['PhysAct'] = mesa_raw['PhysAct'].replace(diet_phys_map)

mesa_raw = mesa_raw.dropna()

# standardize continuous variables
cols_to_standardize = ['nSES','nFavFood','nPhysFac','nRS', 'sbp', 'hdl', 'totchol']
scaler = StandardScaler()
mesa_raw[cols_to_standardize] = scaler.fit_transform(mesa_raw[cols_to_standardize])


mesa_bla_raw = mesa_raw[mesa_raw['race'] == 3]

In [ ]:
jhs_outcome_full = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')
jhs_outcome = jhs_outcome_full[['subjid','event','time']].copy()

jhs_covar = pd.read_csv('../jhs/data/processed/jhs_raw_full.csv')
jhs_covar = jhs_covar[jhs_covar['visit'] == 1]

jhs_raw = pd.merge(jhs_outcome, jhs_covar, on=['subjid'], how='left')

jhs_raw = jhs_raw.drop(jhs_raw.columns[4], axis=1)

jhs_raw['nutrition'] = jhs_raw['nutrition'].replace(diet_phys_map)
jhs_raw['PhysAct'] = jhs_raw['PhysAct'].replace(diet_phys_map)

jhs_impute = jhs_raw.copy()

jhs_raw = jhs_raw.dropna()

# standardize continuous variables
cols_to_standardize = ['nSES','nFavFood','nPhysFac','nRS', 'sbp', 'hdl', 'totchol']
scaler = StandardScaler()
jhs_raw[cols_to_standardize] = scaler.fit_transform(jhs_raw[cols_to_standardize])

### Raw data with missing imputed 

In [3]:
mesa_imputed = pd.read_csv('../data_processed/MESA/Y_BaselineX_imputed_full.csv')

diet_phys_map = {0:1, 1:2, 2:3}
mesa_imputed['nutrition'] = mesa_imputed['nutrition'].replace(diet_phys_map)
mesa_imputed['PhysAct'] = mesa_imputed['PhysAct'].replace(diet_phys_map)

# standardize continuous variables
cols_to_standardize = ['nSES','nFavFood','nPhysFac','nRS', 'sbp', 'hdl', 'totchol']
scaler = StandardScaler()
mesa_imputed[cols_to_standardize] = scaler.fit_transform(mesa_imputed[cols_to_standardize])

mesa_bla_imputed = mesa_imputed[mesa_imputed['race'] == 3]

In [ ]:
# jhs

### Processed data

In [4]:
mesa_pre = pd.read_csv('../data_processed/MESA/Y_BaselineX_processed_full.csv')

diet_phys_map = {0:1, 1:2, 2:3}
mesa_pre['nutrition'] = mesa_pre['nutrition'].replace(diet_phys_map)
mesa_pre['PhysAct'] = mesa_pre['PhysAct'].replace(diet_phys_map)

mesa_bla_pre = mesa_pre[mesa_pre['race'] == 3]

In [ ]:
jhs_pre = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')
jhs_pre = jhs_pre.drop("nSES", axis=1)

jhs_pre['nutrition'] = jhs_pre['nutrition'].replace(diet_phys_map)
jhs_pre['PhysAct'] = jhs_pre['PhysAct'].replace(diet_phys_map)

## Moderation test

In [ ]:
# standardize
# Test alcohol and smoking
# ordinal categorical and treat as continuous variable
# bootstrap to report CI for the interaction coefficient 

In [7]:
named_df_list = [('mesa_raw', mesa_raw), ('mesa_impute', mesa_imputed), 
                 ('mesa_pre', mesa_pre)]
Y_var = 'Y_10y'
X_var = ['nSES','nFavFood','nPhysFac', 'nRS',
         'FamIncome','nutrition', 'PhysAct','currentSmoker','alc',
         'age','gender','race','Diabetes','hdl','totchol','sbp']


for df_name, df in named_df_list:

    ### estimate CI using bootstrapping
    
    # empty lists to store results
    num_bootstrap_samples = 1000
    
    bootstrap_1_nSES_FamIncome = []
    bootstrap_2_nFavFood_FamIncome = []
    bootstrap_3_nPhysFac_FamIncome = []
    bootstrap_4_nRS_FamIncome = []
    bootstrap_5_nFavFood_nutrition = []
    bootstrap_6_nPhysFac_PhysAct = []
        
    bootstrap_results_all = [bootstrap_1_nSES_FamIncome, bootstrap_2_nFavFood_FamIncome, 
                             bootstrap_3_nPhysFac_FamIncome, bootstrap_4_nRS_FamIncome,
                             bootstrap_5_nFavFood_nutrition, bootstrap_6_nPhysFac_PhysAct]
    bootstrap_names = ['bootstrap_1_nSES_FamIncome', 'bootstrap_2_nFavFood_FamIncome', 
                       'bootstrap_3_nPhysFac_FamIncome', 'bootstrap_4_nRS_FamIncome', 
                       'bootstrap_5_nFavFood_nutrition', 'bootstrap_6_nPhysFac_PhysAct']

    for _ in range(num_bootstrap_samples):
        
        #print(_)
        
        # Draw a bootstrap sample
        bootstrap_sample = df.sample(frac=1, replace=True)
        
        # extract X and Y
        y = bootstrap_sample['Y_10y']
        X = bootstrap_sample[X_var]
        
        # dummy coding
        X = pd.get_dummies(X, columns=['gender','race','currentSmoker','alc','Diabetes'], 
                    drop_first=True)
        
        # add interaction term to separate df
        X1 = X.copy()
        X2 = X.copy()
        X3 = X.copy()
        X4 = X.copy()
        X5 = X.copy()
        X6 = X.copy()

        X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
        X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
        X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
        X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
        X5['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
        X6['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']
        
        # fit regression and store results respectively
        dfs = [X1, X2, X3, X4, X5, X6]
        
        for i, df_test in enumerate(dfs):
            logistic_model_with_moderation = sm.Logit(y, sm.add_constant(df_test)).fit(disp=0)
            last_param = logistic_model_with_moderation.params[-1]
            bootstrap_results_all[i].append(last_param)
        

    ### report result
    
    print(f"Moderation results in dataset: {df_name}\n")
    
    report_df = pd.DataFrame(columns=['Bootstrap Result Name', 'Mean', '2.5%', '97.5%'])

    for i, bootstrap_result in enumerate(bootstrap_results_all):
        # Calculate required statistics
        mean = np.mean(bootstrap_result)
        percentile_2_5 = round(np.percentile(bootstrap_result, 2.5), 5)
        percentile_97_5 = round(np.percentile(bootstrap_result, 97.5), 5)

        # Add to the DataFrame
        report_df = report_df.append({
            'Bootstrap Result Name': bootstrap_names[i],
            'Mean': mean,
            '2.5%': percentile_2_5,
            '97.5%': percentile_97_5
        }, ignore_index=True)

    # Print the DataFrame
    print(report_df)

    print("\n" + "="*50 + "\n")  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
Moderation results in dataset: mesa_impute

            Bootstrap Result Name      Mean     2.5%    97.5%
0      bootstrap_1_nSES_FamIncome  0.092923 -0.00775  0.19148
1  bootstrap_2_nFavFood_FamIncome  0.038343 -0.05811  0.14149
2  bootstrap_3_nPhysFac_FamIncome -0.024093 -0.11789  0.07303
3       bootstrap_4_nRS_FamIncome  0.004364 -0.08323   0.0943
4  bootstrap_5_nFavFood_nutrition  0.008897 -0.17787  0.20093
5    bootstrap_6_nPhysFac_PhysAct  0.003511 -0.10515  0.14427


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
12

### Mesa

In [ ]:
y = mesa_raw['Y_10y']

X = mesa_raw[['nSES','nFavFood','nPhysFac', 'nRS',
          'FamIncome','nutrition', 'PhysAct',
          'age','gender','race','currentSmoker','alc','Diabetes','hdl','totchol','sbp']]

X = pd.get_dummies(X, columns=['gender','race','currentSmoker','alc','Diabetes'], 
                    drop_first=True)

X1 = X.copy()
X2 = X.copy()
X3 = X.copy()
X4 = X.copy()
X5 = X.copy()
X6 = X.copy()

# add interaction term
X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
X5['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
X6['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']

In [ ]:
logistic_model_with_moderation = sm.Logit(y, sm.add_constant(X6)).fit()
print(logistic_model_with_moderation.summary())
print("Last coefficient:", logistic_model_with_moderation.params[-1])

## Mediation Test

In [ ]:
named_df_list = [('jhs_raw', jhs_raw), ('jhs_impute', jhs_impute), 
                 ('jhs_pre', jhs_pre)]

Y = 'Y_tot'
X = 'nSES'
M = 'FamIncome'

In [ ]:
for df_name, df in named_df_list:

    ### estimate CI using bootstrapping

    num_bootstrap_samples = 1000
    bootstrap_results = []

    for _ in range(num_bootstrap_samples):
        # Step 1: Draw a bootstrap sample
        bootstrap_sample = df.sample(frac=1, replace=True)

        # Step 2: Regression of M on X
        model_mediator = sm.OLS(bootstrap_sample[M], sm.add_constant(bootstrap_sample[X])).fit()

        # Step 3: Regression of Y on X
        model_outcome = sm.Logit(bootstrap_sample[Y], sm.add_constant(bootstrap_sample[X])).fit(disp=0)

        # Step 4: Regression of Y on both X and M
        model_combined = sm.Logit(bootstrap_sample[Y], sm.add_constant(bootstrap_sample[[X, M]])).fit(disp=0)

        # Calculate the indirect effect for this bootstrap sample
        indirect_effect_bootstrap = model_combined.params[M] * model_outcome.params[X]

        # Store the result
        bootstrap_results.append(indirect_effect_bootstrap)

    # Calculate the confidence interval
    ci_lower = round(np.percentile(bootstrap_results, 2.5),4)
    ci_upper = round(np.percentile(bootstrap_results, 97.5),4)
    
    
    ### estimates using original sample
    
    # Step 1: Regression of M on X
    model_mediator = sm.OLS(df[M], sm.add_constant(df[X])).fit()
    # Step 2: Regression of Y on X; total effect
    model_outcome = sm.Logit(df[Y], sm.add_constant(df[X])).fit(disp=0)
    # Step 3: Regression of Y on both X and M
    model_combined = sm.Logit(df[Y], sm.add_constant(df[[X, M]])).fit(disp=0)
    # Calculate the indirect effect for the original sample
    indirect_effect_original = model_combined.params[M] * model_outcome.params[X]


    ### report result

    print(f"Mediation results using: {df_name}\n")
    
    print("Step 1 - Coefficient for X in M model:", round(model_mediator.params[X],4), 
          "p-value:", round(model_mediator.pvalues[X],4))

    print("Step 2 - Coefficient for X in Y model:", round(model_outcome.params[X],4), 
          "p-value:", round(model_outcome.pvalues[X],4))

    print("Step 3 - Coefficient for M in Y model (controlling for X):", round(model_combined.params[M],4), 
          "p-value:", round(model_combined.pvalues[M],4))

    print("\nIndirect Effect (original sample):", round(indirect_effect_original,4))
    print("Bootstrapped 95% Confidence Interval:", (ci_lower, ci_upper))
    
    print("\n" + "="*50 + "\n")  


In [ ]:
# check single model

model_mediator = sm.Logit(jhs_pre[Y], sm.add_constant(jhs_pre[X])).fit()
model_mediator.summary()